In [7]:
import pandas as pd
import numpy as np

In [8]:
class MobilePhone:
    #konstruktor
    def __init__(self, model, price, antutu_score, dxomark, ppi, battery_capacity):
        self.model = model
        self.price = price
        self.antutu_score = antutu_score
        self.dxomark = dxomark
        self.ppi = ppi
        self.battery_capacity = battery_capacity
    
    #printowanie telefonu
    def __str__(self):
        return f"Model: {self.model}, Price ($): {self.price}, Antutu Score: {self.antutu_score}, DXOMARK: {self.dxomark}, PPI: {self.ppi}, Battery Capacity (mAh): {self.battery_capacity}"

In [9]:
#wczytaj smartfony i utwórz liste z objektami klasy MobilePhone
data = pd.read_csv("smartfony.csv")
phones = []
for index, row in data.iterrows():
    phone = MobilePhone(row['Model'], row['Cena ($)'], row['Antutu Score'], row['DXOMARK'], row['ppi'], row['pojemność baterii (mAh)'])
    phones.append(phone)


In [10]:
#wyswietl telefony
for phone in phones:
    print(phone)

Model: Huawei Mate 40 Pro, Price ($): 907, Antutu Score: 637 017, DXOMARK: 143, PPI: 450, Battery Capacity (mAh): 4 360
Model: Google Pixel 7 Pro, Price ($): 899, Antutu Score: 788 707, DXOMARK: 147, PPI: 512, Battery Capacity (mAh): 5 000
Model: Honor Magic4 Ultimate, Price ($): 1 211, Antutu Score: 1 051 488, DXOMARK: 147, PPI: 460, Battery Capacity (mAh): 4 600
Model: Apple iPhone 14 Pro Max, Price ($): 1 299, Antutu Score: 960 137, DXOMARK: 146, PPI: 460, Battery Capacity (mAh): 4 323
Model: Xiaomi 12S Ultra, Price ($): 908, Antutu Score: 1 061 590, DXOMARK: 136, PPI: 522, Battery Capacity (mAh): 4 860
Model: Samsung Galaxy S22 Ultra, Price ($): 1 199, Antutu Score: 907 693, DXOMARK: 135, PPI: 500, Battery Capacity (mAh): 5 000
Model: ROG Phone 6 PRO, Price ($): 999, Antutu Score: 1 116 637, DXOMARK: 81, PPI: 395, Battery Capacity (mAh): 6 000
Model: Motorola Edge 30 Pro, Price ($): 699, Antutu Score: 979 697, DXOMARK: 101, PPI: 393, Battery Capacity (mAh): 4 800
Model: Galaxy Z Fo